In [1]:
from groq import Groq
import os
import subprocess
import dotenv

dotenv.load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [2]:

# Path to your image
image_path = "./Report_Images/Complete-blood-count-CBC-results.png"

client = Groq()
client.api_key = GROQ_API_KEY

In [5]:
!pip show pydantic

Name: pydantic
Version: 2.10.6
Summary: Data validation using Python type hints
Home-page: https://github.com/pydantic/pydantic
Author: 
Author-email: Samuel Colvin <s@muelcolvin.com>, Eric Jolibois <em.jolibois@gmail.com>, Hasan Ramezani <hasan.r67@gmail.com>, Adrian Garcia Badaracco <1755071+adriangb@users.noreply.github.com>, Terrence Dorsey <terry@pydantic.dev>, David Montague <david@pydantic.dev>, Serge Matveenko <lig@countzero.co>, Marcelo Trylesinski <marcelotryle@gmail.com>, Sydney Runkle <sydneymarierunkle@gmail.com>, David Hewitt <mail@davidhewitt.io>, Alex Hall <alex.mojaki@gmail.com>, Victorien Plot <contact@vctrn.dev>
License-Expression: MIT
Location: e:\D\Work\Projects\Medical Lab Bot\Test_Files\.venv\Lib\site-packages
Requires: annotated-types, pydantic-core, typing-extensions
Required-by: Crawl4AI, fastapi, groq, langchain, langchain-core, langsmith, litellm, openai, pydantic-settings


In [16]:
import base64
import re
import json
import pandas as pd
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from typing import List, Dict

# Define Pydantic model for structured output (Pydantic V2)
class LabReport(BaseModel):
    test_name: str = Field(description="The name of the medical lab test (e.g., Complete Blood Count, Lipid Profile)")
    table_data: List[Dict[str, str]] = Field(description="List of dictionaries containing table rows with Parameter, Initial_Value, Later_Value, and Unit")

def process_image(chat_instance, image_path):
    # Function to encode the image
    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # Getting the base64 string
    base64_image = encode_image(image_path)

    def request_llm(base64_image, attempt=1):
        # Define parser for structured output
        parser = PydanticOutputParser(pydantic_object=LabReport)
        
        # Prompt for LaTeX table output
        prompt = f"""Extract the medical lab report details from the provided image and format them as a LaTeX table. Include the test name as the table caption and the table data with columns for Parameter, Initial_Value, Later_Value, and Unit. Ignore all unnecessary information like patient name, address, or anything else not related to the lab data. Ensure accurate readings and units from the image. Return ONLY the LaTeX table code, starting with \\begin{{table}} and ending with \\end{{table}}, with no additional text or explanations. The table structure should align with this format: {parser.get_format_instructions()}"""
        
        # Modify prompt for second attempt
        if attempt > 1:
            prompt = f"""Extract the medical lab report information from the image, focusing only on the test name and report data. Ignore irrelevant details like patient name or address. Format the output as a LaTeX table with the test name as the caption and columns for Parameter, Initial_Value, Later_Value, and Unit. Ensure accurate readings and units. Return ONLY the LaTeX table code, starting with \\begin{{table}} and ending with \\end{{table}}, with no additional text. The table structure should align with: {parser.get_format_instructions()}"""

        chat_completion = chat_instance.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
                    ],
                }
            ],
            model="llama-3.2-90b-vision-preview",
        )
        
        return chat_completion.choices[0].message.content
    
    # Get response
    response = request_llm(base64_image)
    
    # Parse response to validate LaTeX table
    latex_table = parse_response_to_latex(response)
    
    # Retry with modified prompt if parsing fails
    if not latex_table:
        response = request_llm(base64_image, attempt=2)
        latex_table = parse_response_to_latex(response)
    
    return latex_table

def parse_response_to_latex(response: str) -> str:
    """
    Parse the model response to extract and validate a LaTeX table.
    
    Args:
        response (str): Raw response from the model.
    
    Returns:
        str: LaTeX table code or empty string if parsing fails.
    """
    try:
        # Extract LaTeX table
        pattern = re.compile(r'\\begin{table}.*?\\end{table}', re.DOTALL)
        match = pattern.search(response)
        if not match:
            raise ValueError("No valid LaTeX table found")
        
        latex_table = match.group(0).strip()
        
        # Validate structure by converting to JSON-like format for Pydantic
        # (This ensures the table data matches the expected format)
        lines = latex_table.split('\n')
        test_name = ""
        table_data = []
        in_tabular = False
        headers = ["Parameter", "Initial_Value", "Later_Value", "Unit"]
        
        for line in lines:
            line = line.strip()
            if line.startswith("\\caption{"):
                test_name = line[len("\\caption{"):-1]
            if line.startswith("\\begin{tabular}"):
                in_tabular = True
                continue
            if line.startswith("\\end{tabular}"):
                in_tabular = False
                continue
            if in_tabular and not line.startswith("\\hline") and not line.startswith("Parameter"):
                # Parse table row
                parts = [p.strip() for p in line.split('&')]
                if len(parts) >= 4:
                    row = {
                        "Parameter": parts[0].replace("\\", ""),
                        "Initial_Value": parts[1].replace("\\", ""),
                        "Later_Value": parts[2].replace("\\", ""),
                        "Unit": parts[3].split('\\\\')[0].replace("\\", "")
                    }
                    table_data.append(row)
        
        # Validate with Pydantic
        lab_report = LabReport(test_name=test_name, table_data=table_data)
        lab_report.model_dump()  # Ensure validation
        
        return latex_table
    except Exception as e:
        print(f"Error parsing LaTeX table: {e}")
        return ""

# Example usage
# from groq import Groq
# client = Groq(api_key="your-groq-api-key")
# latex_table = process_image(client, "path/to/lab_report.jpg")
# print(latex_table)

In [17]:
# Example usage
# from groq import Groq
# client = Groq(api_key="your-api-key")
df = process_image(client, image_path)
print(df)

\begin{table}
  \begin{center}
    \begin{tabular}{|c|c|c|c|}
      \hline
      Parameter & Initial Value & Later Value & Unit \\
      \hline
      White blood cells (WBC) & 1.90 & 4.25 & K/μL \\
      Red blood cells (RBC) & 3.75 & 2.47 & M/μL \\
      Hemoglobin (Hb) & 11 & 8.8 & g/dL \\
      Hematocrit (Hct) & 30.6 & 26.9 & \% \\
      Mean cell volume (MCV) & 81.6 & 108.9 & fL \\
      Mean cell hemoglobin (MCH) & 29.3 & 35.6 & pg \\
      Platelets & 12 & 51 & K/μL \\
      Reticulocyte absolute & 0.020 & 0.128 & \% \\
      \hline
    \end{tabular}
  \end{center}
  \caption{Laboratory Test Results}
\end{table}
